In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.optimize import minimize
from sklearn.preprocessing import MinMaxScaler

In [2]:
# constants
SUBFIX = "DEPTH/raw"
BASEDIR = "data/image_data"
H = 480
W = 640

ANCHOR_POINT = [225, 264]


In [3]:
distance_folders = [f for f in os.listdir(BASEDIR) if os.path.isdir(os.path.join(BASEDIR, f))]
distances = [int(d.split("_")[0]) for d in distance_folders]

In [4]:
def load_raw(path, h, w):
    data = np.fromfile(path, dtype=np.uint16)
    return data.reshape((h, w))

def crop_center(array, crop_size):
    if array.ndim != 2:
        raise ValueError("Input array must be a 2D array")
    height, width = array.shape
    center_y, center_x = height // 2, width // 2
    half_crop_size = crop_size // 2

    # Calculate start and end indices
    start_y = max(0, center_y - half_crop_size)
    end_y = min(height, center_y + half_crop_size)
    start_x = max(0, center_x - half_crop_size)
    end_x = min(width, center_x + half_crop_size)

    # Crop and return the central square
    return array[start_y:end_y, start_x:end_x]

In [5]:
test_data = load_raw("data/image_data/155_N09ASH24DH0012_2024_05_06_18_46_01/DEPTH/raw/Depth-2024-5-6-18-46-2-68-1-1550-210107604.raw", H, W)

In [6]:
valid_data = test_data[ANCHOR_POINT[0]-25:ANCHOR_POINT[0]+25, ANCHOR_POINT[1]-25:ANCHOR_POINT[1]+25]
print(valid_data.shape)
mean_dist = np.mean(valid_data)
mean_dist

(50, 50)


150.3248

In [7]:
dist_dict = {}
dist_list = []
for folder in distance_folders:
    distance = folder.split("_")[0]
    rawpath = os.path.join(BASEDIR, folder, SUBFIX)
    paths = [f for f in os.listdir(rawpath) if os.path.isfile(os.path.join(rawpath, f))]
    mean_dist_holder = []
    for path in paths:
        path = os.path.join(rawpath, path)
        raw = load_raw(path, H, W)
        valid_raw = raw[ANCHOR_POINT[0]-25:ANCHOR_POINT[0]+25, ANCHOR_POINT[1]-25:ANCHOR_POINT[1]+25]
        mu = np.mean(valid_raw)
        mean_dist_holder.append(mu)
    final_mu = np.mean(mean_dist_holder)
    dist_dict[distance] = final_mu
    dist_list.append(final_mu)


In [8]:
df = pd.read_csv("data/dq_0507.csv")
df

,Unnamed: 0,actual_depth,fit_depth,error_percentage,absolute_error,baseline,focal
0,0,106,102.0,-3.773585,-4.0,52.916119,166.366119
1,1,155,148.7,-4.064516,-6.3,52.916119,166.366119
2,2,207,202.5,-2.173913,-4.5,52.916119,166.366119
3,3,250,245.3,-1.880000,-4.7,52.916119,166.366119
4,4,300,295.7,-1.433333,-4.3,52.916119,166.366119
5,5,350,349.1,-0.257143,-0.9,52.916119,166.366119
6,6,400,399.1,-0.225000,-0.9,52.916119,166.366119
7,7,453,450.6,-0.529801,-2.4,52.916119,166.366119
8,8,499,493.5,-1.102204,-5.5,52.916119,166.366119
9,9,548,549.5,0.273723,1.5,52.916119,166.366119


In [9]:
df["avg_depth_50x50_anchor"] = df['actual_depth'].astype(str).map(dist_dict)

In [10]:
df

,Unnamed: 0,actual_depth,fit_depth,error_percentage,absolute_error,baseline,focal,avg_depth_50x50_anchor
0,0,106,102.0,-3.773585,-4.0,52.916119,166.366119,102.158987
1,1,155,148.7,-4.064516,-6.3,52.916119,166.366119,150.414320
2,2,207,202.5,-2.173913,-4.5,52.916119,166.366119,203.721600
3,3,250,245.3,-1.880000,-4.7,52.916119,166.366119,244.228133
4,4,300,295.7,-1.433333,-4.3,52.916119,166.366119,293.806160
5,5,350,349.1,-0.257143,-0.9,52.916119,166.366119,348.504133
6,6,400,399.1,-0.225000,-0.9,52.916119,166.366119,399.523307
7,7,453,450.6,-0.529801,-2.4,52.916119,166.366119,445.513520
8,8,499,493.5,-1.102204,-5.5,52.916119,166.366119,489.437813
9,9,548,549.5,0.273723,1.5,52.916119,166.366119,549.873280


In [14]:
df['avg_disp_50x50_anchor'] = 52.916119 * 166.366119 / df['avg_depth_50x50_anchor']
df['actual_disp'] = 52.916119 * 166.366119	 / df['actual_depth']
df['fit_disp'] = 52.916119 * 166.366119	 / df['fit_depth']


In [15]:
df

,Unnamed: 0,actual_depth,fit_depth,error_percentage,absolute_error,baseline,focal,avg_depth_50x50_anchor,avg_disp_50x50_anchor,actual_disp,fit_disp
0,0,106,102.0,-3.773585,-4.0,52.916119,166.366119,102.158987,86.174008,83.051409,86.308327
1,1,155,148.7,-4.064516,-6.3,52.916119,166.366119,150.414320,58.528000,56.796447,59.202753
2,2,207,202.5,-2.173913,-4.5,52.916119,166.366119,203.721600,43.213137,42.528741,43.473824
3,3,250,245.3,-1.880000,-4.7,52.916119,166.366119,244.228133,36.046008,35.213797,35.888501
4,4,300,295.7,-1.433333,-4.3,52.916119,166.366119,293.806160,29.963461,29.344831,29.771557
5,5,350,349.1,-0.257143,-0.9,52.916119,166.366119,348.504133,25.260674,25.152712,25.217558
6,6,400,399.1,-0.225000,-0.9,52.916119,166.366119,399.523307,22.034883,22.008623,22.058254
7,7,453,450.6,-0.529801,-2.4,52.916119,166.366119,445.513520,19.760229,19.433663,19.537171
8,8,499,493.5,-1.102204,-5.5,52.916119,166.366119,489.437813,17.986860,17.642183,17.838803
9,9,548,549.5,0.273723,1.5,52.916119,166.366119,549.873280,16.009960,16.064689,16.020836


In [16]:
df.to_csv('data/dq_0509.csv', index=False)